# Rappel du jour 2 

* tests
    * pyramide de tests
* manipulations de types / structures de données
    * utilisation de `fstrings`
    * dictionnaires et sets 
* début de programmation fonctionnelle
    * les fonctions sont des objets comme les autres 

In [8]:
def toto():
    global tata 
    tata += 1 
    print(id(tata), tata)

In [9]:
tata = 3.234432
print(id(tata), tata)
toto()
print(id(tata), tata)

4429620304 3.234432
4429617904 4.234432
4429617904 4.234432


In [15]:
fs = []
for i in range(5):
    def f(x, nombre=i):
        return x + nombre
    fs.append(f)

print(i)
for f in fs:
    print(f(10))

4
10
11
12
13
14


In [21]:
def spy_list():
    print(1)
    return []

def append_wrong(val, li=spy_list()):
    print(2)
    li.append(val)
    return li

print(3)

1
3


In [23]:
append_wrong(20)

2


[2, 20]